# Preprocessing the dataset

While the dataset is easily accessable from smartdublin and met eireann's websites, the data must be correctly preprocessed before use.

Imports for this part

In [5]:
import os
import time
import warnings
import zipfile
import wget
from pandas.errors import DtypeWarning
from tqdm import tqdm
import pandas as pd
from os.path import isfile, join
import numpy as np
from datetime import datetime, timedelta
from numpy import nan
import csv
import math

Downloads the data from their respective websites

In [6]:
def dublin_weather():
    """
    Download dublin weather data from met eireann
    """

    destination = '../datasets/weather'
    url = "https://cli.fusio.net/cli/climate_data/webdata/hly175.zip"

    if not os.path.exists(destination):
        os.makedirs(destination)

    # Download if file does not exist
    # Downloads as zip, so code exists to unzip and remove it
    if not os.path.isfile(destination + "/hly175/hly175.csv"):
        print("Downloading hourly Dublin weather data")
        wget.download(url, destination)
        with zipfile.ZipFile(destination + "/hly175.zip", "r") as zip_ref:
            zip_ref.extractall(destination + "/hly175")
        os.remove(destination + "/hly175.zip")
    else:
        print("Dublin weather data already exists\n")
        
dublin_weather()

Dublin weather data already exists



In [7]:
def dublin_bss():
    """
    Download dublin bss data with wget
    """
    destination = '../datasets/bss/dublin/original/'

    if not os.path.exists(destination):
        os.makedirs(destination)

    base_url = "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource"
    urls = ["/99a35442-6878-4c2d-8dff-ec43e91d21d7/download/dublinbikes_20200701_20201001.csv",
            "/8ddaeac6-4caf-4289-9835-cf588d0b69e5/download/dublinbikes_20200401_20200701.csv",
            "/aab12e7d-547f-463a-86b1-e22002884587/download/dublinbikes_20200101_20200401.csv",
            "/5d23332e-4f49-4c41-b6a0-bffb77b33d64/download/dublinbikes_20191001_20200101.csv",
            "/305d39ac-b6a0-4216-a535-0ae2ddf59819/download/dublinbikes_20190701_20191001.csv",
            "/76fdda3d-d8be-441b-92dd-0ee36d9c5316/download/dublinbikes_20190401_20190701.csv",
            "/538165d7-535e-4e1d-909a-1c1bfae901c5/download/dublinbikes_20190101_20190401.csv",
            "/67ea095f-67ad-47f5-b8f7-044743043848/download/dublinbikes_20181001_20190101.csv",
            "/9496fac5-e4d7-4ae9-a49a-217c7c4e83d9/download/dublinbikes_20180701_20181001.csv",
            "/5328239f-bcc6-483d-9c17-87166efc3a1a/download/dublinbikes_20201001_20210101.csv",
            "/2dec86ed-76ed-47a3-ae28-646db5c5b965/download/dublin.csv"]

    print("Downloading dublinbikes data")
    for url in tqdm(urls):
        filename = url.split("/")[-1]
        if filename in os.listdir(destination):
            # print("File \"" + filename + "\" already exists")
            continue

        final_url = base_url + url
        # print("starting download on " + filename)
        wget.download(final_url, destination)
        # print(filename + " : downloaded\n")

    print("Finished downloading Dublin BSS data")

    # Clear generated tmp files
    for file in os.listdir():
        if file[-4:] == ".tmp":
            os.remove(file)
            
dublin_bss()

100%|████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 3659.95it/s]

Finished downloading Dublin BSS data


In [8]:
def get_station_ids():
    # Get all the station IDs
    dataset = pd.read_csv('../datasets/bss/dublin/original/dublin.csv',
                          usecols=['Number'])
    station_ids = []
    for d in dataset['Number'].unique():
        station_ids.append(str(d))
    station_ids.sort()
    return station_ids

This function reorganizes all the data into CSVs sorted by station instead of yearly quarter. This is to make the data easier to work with, as the size of the dataset means that it is easiet to process the data on a station-by-station basis. This part also removes columns that are either redundant (latitude/longitude) or do not add anything to the analysis ("LAST UPDATED")

In [9]:
def convert_time(x):
    """
    Converts TIME field in the CSV to an integer representing
    what time of day it is (in number of 5min increments) from 0 to 287
    eg
    - 00:00 -> 0
    - 00:10 -> 2
    - 02:20 -> 28
    etc
    """
    a = x.split(' ')
    a = a[1].split(':')

    ans = math.floor((int(a[0]) * 12) + (int(a[1]) / 5))

    return ans

def convert_date(x):
    """
    Converts TIME field to an integer representing the day of the year
    
    eg
    - 2019-02-10 -> 41
    """
    current_date = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    return current_date.strftime('%j')

def convert_day(x):
    """
    Converts TIME field to an integer representing the day of the year
    
    eg
    - 2019-02-10 -> 41
    """
    current_date = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    return current_date.strftime('%w')
    

def organise_by_station():
    """
    Reorganise all the dublinbikes CSVs by station instead of by quarter
    """
    # Get list of data files
    bss_files = os.listdir('../datasets/bss/dublin/original')
    if 'dublin.csv' in bss_files:
        bss_files.remove('dublin.csv')

    if 'reorg' in bss_files:
        bss_files.remove('reorg')

    # Get all the station IDs
    dataset = pd.read_csv('../datasets/bss/dublin/original/dublin.csv',
                          usecols=['Number'])
    station_ids = []
    for d in dataset['Number'].unique():
        station_ids.append(d)
    station_ids.sort()

    # Get column names
    columns = pd.read_csv('../datasets/bss/dublin/original/dublinbikes_20200701_20201001.csv', nrows=1).columns

    # Create the directory if it does not exist
    destination = '../datasets/bss/dublin/reorg/'
    if not os.path.exists(destination):
        os.makedirs(destination)

    print("Starting dublinbikes reorganisation")
    
    for station in tqdm(station_ids):
        if os.path.exists('../datasets/bss/dublin/reorg/station_' + str(station) + '.csv'):
            # print('\tStation CSV already exists')
            continue

#         print('Working on station: ' + str(station))
        df1 = pd.DataFrame(columns=columns)
        for file in bss_files:
            df2 = pd.read_csv('../datasets/bss/dublin/original/' + str(file))
            df2 = df2[df2['STATION ID'] == station]
            temp = [df1, df2]
            df1 = pd.concat(temp)
        df1 = df1.drop(df1.columns[[0]], axis=1)
        df1 = df1.drop([
            'LAST UPDATED',
            'NAME',
            'BIKE STANDS',
            'AVAILABLE BIKE STANDS',
            'STATUS',
            'ADDRESS',
            'LATITUDE',
            'LONGITUDE'
        ], axis=1)
        
        df1['int_time'] = df1['TIME'].apply(lambda x: convert_time(x))
        df1['int_date'] = df1['TIME'].apply(lambda x: convert_date(x))
        df1['int_day'] = df1['TIME'].apply(lambda x: convert_day(x))
        
        # Remove duplicate april 1st
        df1 = df1.drop_duplicates()
        
        df1.to_csv('../datasets/bss/dublin/reorg/station_' + str(station) + '.csv', index=False)

    print('\nFinished dublinbikes reorganisation')
    
organise_by_station()


100%|██████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 3438.11it/s]

Starting dublinbikes reorganisation

Finished dublinbikes reorganisation


The first 15 rows in the downloaded CSV contains information describing the dataset, this is removed before processing to allow us to use the dataset with pandas. Much like the station data I also remove some columns that have no use for the project (like msl which is"Mean Sea Level Pressure").

In [10]:
def clean_weather_data():
    # Remove first n rows, that contain data from before the start of the BSS data
    with open('../datasets/weather/hly175/hly175.csv', 'r') as fin:
        data = fin.read().splitlines(True)
    with open('../datasets/weather/hly175/hly175clean.csv', 'w') as fout:
        fout.writelines(data[15])
        fout.writelines(data[131164:])

    # Remove all rows with non-useful data
    warnings.filterwarnings("ignore", category=DtypeWarning)
    dataset = pd.read_csv('../datasets/weather/hly175/hly175clean.csv',
                          usecols=['date', 'rain', 'temp', 'rhum'])

    #Was used for an old approach for joining weather and station data
    #dataset["epoch"] = dataset["date"].apply(lambda x: int(time.mktime(time.strptime(x, "%d-%b-%Y %H:%M"))))

    dataset.to_csv('../datasets/weather/hly175/hly175clean.csv', index=False)
    
clean_weather_data()

Many rows in the station data is missing, this function replcaes them with null rows. This is necessary to ensure that the weather data lines up correctly

In [11]:
def add_null_rows(station_number):
    
    station_file = "../datasets/bss/dublin/reorg/station_" + station_number + ".csv"
    
    if not os.path.exists(station_file):
        # print("File: " + str(station_file) + " does not exist")
        return

    # Create the directory if it does not exist
    destination = '../datasets/bss/dublin/reorg_w_null/'
    if not os.path.exists(destination):
        os.makedirs(destination)

    if os.path.exists(destination + station_file.split("/")[-1]):
        # print("File plus weather already exists")
        return

    station = pd.read_csv(station_file)

    start_time = datetime.strptime(station['TIME'].iloc[0][:-3], "%Y-%m-%d %H:%M")

    pos = 0
    with open(destination + '/' + station_file.split('/')[-1], 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = station.columns
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        empty_row = {}
        current_row = {}
        for c in station.columns:
            empty_row[c] = nan
            current_row[c] = nan


        #         for index, row in tqdm(station.iterrows(), total=station.shape[0]):
        for index, row in station.iterrows():

            target_time = start_time + timedelta(minutes=pos * 5)
            str_time = row['TIME'][:-3]

            if int(str_time[-1]) >= 5:
                str_time = str_time[:-1] + '5'
            else:
                str_time = str_time[:-1] + '0'

            current_time = datetime.strptime(str_time, "%Y-%m-%d %H:%M")
            while current_time > target_time:
                writer.writerow(empty_row)

                pos = pos + 1
                target_time = start_time + timedelta(minutes=pos * 5)

            for c in station.columns:
                current_row[c] = row[c]

            # writer.writerow({
            #     'TIME': row['TIME'],
            #     'BIKE STANDS': row['BIKE STANDS'],
            #     'AVAILABLE BIKE STANDS': row['AVAILABLE BIKE STANDS'],
            #     'AVAILABLE BIKES': row['AVAILABLE BIKES'],
            # })
            writer.writerow(current_row)
            pos = pos + 1
            
station_ids = get_station_ids()
for station in tqdm(station_ids):
    add_null_rows(station)

100%|██████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 2245.07it/s]


In [12]:
def attach_weather_data(station_number):
    
    source_file = "../datasets/bss/dublin/reorg_w_null/station_" + station_number + ".csv"
    
    if not os.path.exists(source_file):
#         print("File " + str(source_file) + " does not exist in /reorg_w_null")
        return

    station_dataframe = pd.read_csv(source_file)

    destination_file = '../datasets/bss/dublin/reorg_plus_weather/station_' + station_number + '.csv'

    destination_directory = '../datasets/bss/dublin/reorg_plus_weather/'

    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    if os.path.exists(destination_file):
        # print("File plus weather already exists")
        return

    weather = pd.read_csv('../datasets/weather/hly175/hly175clean.csv')

    new_weather_dataframe = pd.DataFrame(np.repeat(weather.values, 12, axis=0), columns=weather.columns)
    
    result = pd.concat([station_dataframe, new_weather_dataframe], axis=1)
    
    result = result.drop(['date'], axis=1)
    
    result = result.dropna()

    result.to_csv('../datasets/bss/dublin/reorg_plus_weather/station_' + station_number + '.csv', index=False)

# mypath = "./datasets/bss/dublin/reorg_w_null/"

# files = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]

# for file in tqdm(files):
#     attach_weather_data(mypath, file)
    
station_ids = get_station_ids()
for station in tqdm(station_ids):
    attach_weather_data(station)

# attach_weather_data(mypath, "station_3.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 110/110 [00:43<00:00,  2.52it/s]
